## Bu calisma ~/tf-1 virtaul env ile calisiyor
## tf version 1.15.2

In [1]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_hub as hub

In [100]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [101]:
tf.__version__

'1.15.2'

In [120]:
pd.set_option('display.max_colwidth', -1)
## For multiple outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [102]:
url = 'https://tfhub.dev/google/universal-sentence-encoder-large/3'
embed = hub.Module(url) 

In [103]:
df = pd.read_csv("../Spam_filter/spam.csv", encoding = "ISO-8859-1")

In [104]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.rename(columns={'v1':'label', 'v2':'text'}, inplace=True)

In [105]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [152]:
df.shape

(5572, 2)

In [106]:
X = list(df['text'])
y = list(df['label'])

In [141]:
le = LabelEncoder()

In [142]:
le.fit(y)
le.classes_

LabelEncoder()

array(['ham', 'spam'], dtype='<U4')

In [133]:
# a = ['ankara', 'bursa', 'izmir', 'ora']

# # to categorical
# a_enc = le.fit_transform(a)
# a_enc

# # to one hot encode
# keras.utils.to_categorical(a_enc)

array([0, 1, 2, 3])

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [147]:
def encode(le, labels):
    # we're coverting lables into numbers
    enc = le.transform(labels)
    # with keras.utils, we're converting numbers into one hot encoding 
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [148]:
test = encode(le, ['ham','spam', 'ham', 'spam'])
test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [150]:
untest = decode(le, test)
untest

array(['ham', 'spam', 'ham', 'spam'], dtype='<U4')

In [151]:
X_enc = X
y_enc = encode(le, y)

In [180]:
X_train = np.asarray(X_enc[:5000])
y_train = np.asarray(y_enc[:5000])

X_test = np.asarray(X_enc[5000:])
y_test = np.asarray(y_enc[5000:])

In [157]:
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model

In [159]:
import tensorflow.keras.backend as K

In [161]:
def UniversalEmbedding(X):
    return embed(tf.squeeze(tf.cast(X, tf.string)))

In [166]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512,))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model= Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [171]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(X_train, y_train, epochs=5, batch_size=32)
    model.save_weights('./model.h5')

Train on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 37s 7ms/sample - loss: 0.1479 - acc: 0.9500
Epoch 2/5
5000/5000 [==============================] - 36s 7ms/sample - loss: 0.0564 - acc: 0.9810
Epoch 3/5
5000/5000 [==============================] - 35s 7ms/sample - loss: 0.0479 - acc: 0.9856
Epoch 4/5
5000/5000 [==============================] - 38s 8ms/sample - loss: 0.0423 - acc: 0.9860
Epoch 5/5
5000/5000 [==============================] - 37s 7ms/sample - loss: 0.0370 - acc: 0.9880


In [174]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')
    predicts = model.predict(X_test, batch_size=32)    

In [178]:
predicts

array([[9.9999869e-01, 1.3352968e-06],
       [9.9999118e-01, 8.8777106e-06],
       [1.4261702e-03, 9.9857378e-01],
       ...,
       [9.9678952e-01, 3.2104680e-03],
       [9.9994874e-01, 5.1213334e-05],
       [9.9736148e-01, 2.6385062e-03]], dtype=float32)

In [181]:
y_test

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [182]:
y_test = decode(le, y_test)

In [183]:
y_test

array(['ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam',
       'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham'

In [186]:
y_preds = decode(le, predicts)
y_preds

array(['ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam',
       'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'h

In [187]:
from sklearn import metrics

In [188]:
metrics.confusion_matrix(y_test, y_preds)

array([[493,   5],
       [  5,  69]])

In [190]:
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       498
        spam       0.93      0.93      0.93        74

    accuracy                           0.98       572
   macro avg       0.96      0.96      0.96       572
weighted avg       0.98      0.98      0.98       572

